In [8]:
import numpy as np
import pandas as pd
from pathlib import Path

In [9]:
csv_path = Path("d:/Downloads/tracks/mask-rcnn_tracks.csv")
assert csv_path.exists()

In [10]:
train_fold = ['BARCELONA', 'BRNO', 'ERFURT', 'KAUNAS', 'LEIPZIG', 'NUREMBERG', 'PALMA', 'PRAGUE', 'TALLINN', 'TARTU', 'VILNIUS', 'WEIMAR']
df = pd.read_csv(csv_path)
labeled_df = df[(df['labeled'] == 1) & (df['filename'].isin(train_fold))]

In [11]:
labeled_df[::-1]

,vid,filename,frame_num,track,cx,cy,w,h,track_length,labeled,requires_features
1477470,clip_000034.mp4,WEIMAR,539,26,754.0,422.0,16,50,132,1,1
1477469,clip_000034.mp4,WEIMAR,538,26,753.0,422.5,16,49,131,1,1
1477468,clip_000034.mp4,WEIMAR,537,26,753.0,422.5,16,49,130,1,1
1477467,clip_000034.mp4,WEIMAR,536,26,752.5,421.5,15,49,129,1,1
1477466,clip_000034.mp4,WEIMAR,535,26,752.5,421.0,15,48,128,1,1
...,...,...,...,...,...,...,...,...,...,...,...
614,clip_000000.mp4,BARCELONA,33,13,960.5,646.5,21,39,33,1,1
613,clip_000000.mp4,BARCELONA,32,13,957.5,645.5,21,39,32,1,1
612,clip_000000.mp4,BARCELONA,31,13,955.5,645.5,21,39,31,1,1
611,clip_000000.mp4,BARCELONA,30,13,952.5,645.5,21,41,30,1,1


In [12]:
total_data = np.empty((0, 90, 4))

try:
    for label in labeled_df.iterrows():
        data : pd.Series = label[1]
        filename = data.get('filename')
        vid = data.get('vid')
        track = data.get('track')
        frame_num = data.get('frame_num')

        data = df.loc[(df['filename'] == filename) & 
                    (df['vid'] == vid) & 
                    (df['track'] == track) &
                    (df['frame_num'] <= frame_num + 60) & 
                    (df['frame_num'] >= frame_num - 29), 
                    ['cx', 'cy',	'w', 'h']].to_numpy()

        data = data.reshape((1, 90, 4))
        total_data = np.concatenate([total_data, data])
except:
    import traceback

    traceback.print_exc()
finally:
    np.save('tracks_rev_all.npy', total_data)

Traceback (most recent call last):
  File "C:\Users\Wen Kang\AppData\Local\Temp\ipykernel_5228\1899420853.py", line 11, in <module>
    data = df.loc[(df['filename'] == filename) &
  File "c:\Users\Wen Kang\Desktop\final_year_project\lib\site-packages\pandas\core\ops\common.py", line 72, in new_method
    return method(self, other)
  File "c:\Users\Wen Kang\Desktop\final_year_project\lib\site-packages\pandas\core\arraylike.py", line 72, in __and__
    return self._logical_method(other, operator.and_)
  File "c:\Users\Wen Kang\Desktop\final_year_project\lib\site-packages\pandas\core\series.py", line 6254, in _logical_method
    res_values = ops.logical_op(lvalues, rvalues, op)
  File "c:\Users\Wen Kang\Desktop\final_year_project\lib\site-packages\pandas\core\ops\array_ops.py", line 395, in logical_op
    res_values = na_logical_op(lvalues, rvalues, op)
  File "c:\Users\Wen Kang\Desktop\final_year_project\lib\site-packages\pandas\core\ops\array_ops.py", line 305, in na_logical_op
    res

In [13]:
parsed = np.load("tracks_rev_all.npy")

In [14]:
parsed.shape

(11589, 90, 4)

In [58]:
input_x = parsed[:, :30, :]
target_y = parsed[:, 30:, :]

In [60]:
def convert_cxcywh_to_xywh(bbox):
    converted_bbox = bbox.copy()
    converted_bbox[..., 0] = bbox[..., 0] - bbox[..., 2] / 2
    converted_bbox[..., 1] = bbox[..., 1] - bbox[..., 3] / 2
    return converted_bbox

In [62]:
convert_cxcywh_to_xywh(input_x[[0]])

array([[[889., 626.,  22.,  38.],
        [891., 626.,  23.,  39.],
        [893., 626.,  23.,  41.],
        [895., 626.,  24.,  42.],
        [897., 626.,  23.,  40.],
        [899., 626.,  22.,  40.],
        [900., 626.,  24.,  42.],
        [901., 626.,  24.,  42.],
        [903., 626.,  22.,  41.],
        [904., 626.,  22.,  41.],
        [905., 627.,  22.,  40.],
        [905., 626.,  22.,  39.],
        [906., 626.,  21.,  38.],
        [906., 626.,  22.,  39.],
        [907., 625.,  21.,  40.],
        [908., 625.,  21.,  39.],
        [909., 625.,  21.,  39.],
        [910., 625.,  21.,  39.],
        [912., 625.,  20.,  39.],
        [913., 624.,  21.,  40.],
        [914., 624.,  21.,  40.],
        [915., 625.,  22.,  41.],
        [918., 626.,  21.,  41.],
        [920., 626.,  21.,  40.],
        [923., 626.,  21.,  41.],
        [925., 626.,  22.,  41.],
        [928., 626.,  22.,  41.],
        [932., 625.,  21.,  41.],
        [935., 625.,  21.,  41.],
        [939.,

In [64]:
from torch.utils.data import TensorDataset
import torch

In [65]:
dataset = TensorDataset(torch.from_numpy(input_x), torch.from_numpy(target_y))